In [61]:
import pandas as pd
import numpy as np
import sqlalchemy
import pymssql
import itertools
from collections import defaultdict
from difflib import SequenceMatcher
import time
import helper_functions
import config


def updateQuery(df, engine):
    ###Create temp table and then update only the rows that belong to temp AND product###
    # df.to_sql("temp_table", schema='%s.dbo' % dbName, con=engine, if_exists='replace', index=False)
    df.to_sql("temp_table", con = engine, if_exists = 'replace', index = False)
    with engine.begin() as conn:
        conn.execute(config.UPDATESQLQUERY)

#Relevant similarity function#

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def contains_word(s, w):
    return f' {w} ' in f' {s} '

def editStrings(s):
    ###Check for specif values of s and change the values to match those of Energiers###
    s = (s[0], 'V NECK', s[2], s[3]) if (s[1] == 'V-NECK' and attr == 'NeckDesign') else s
    s = (s[0], 'OFF SHOULDER', s[2], s[3]) if (s[1] == 'OFF-SHOULDER' and attr == 'NeckDesign') else s
    s = (s[0], s[1] + " LENGTH", s[2], s[3]) if (s[1] == 'SHORT' or s[1] == 'MEDIUM' or s[1] == 'KNEE' and attr == 'Length') else s
    s = (s[0], 'LONG', s[2], s[3]) if (s[1] == 'MAXI' and attr == 'Length') else s
    s = (s[0], s[1] + " COLLAR", s[2], s[3]) if ((s[1] == 'MAO' or s[1] == 'STAND UP' or s[1] == 'POLO') and attr == 'CollarDesign') else s
    s = (s[0], s[1] + " FIT", s[2], s[3]) if ((s[1] == 'REGULAR' or s[1] == 'RELAXED' or s[1] == 'SLIM') and attr == 'Fit') else s
    return s


In [59]:
if __name__ == "__main__":
    # Begin Counting Time
    start_time = time.time() 
    ### Read Table Products from S4F database ###
    print('Loading Product table...')
    currendDir = helper_functions.TEXT_MINING
    engine = helper_functions.ENGINE
    dbName = helper_functions.DB_NAME

    # productsDF = pd.read_sql_query('SELECT * FROM  %s.dbo.Product' % dbName, engine)
    productsDF = pd.read_sql_query('SELECT * FROM public.\"Product\"', engine)

    labelsDF = pd.DataFrame()
    
    ### Filter the old elements with the new one, Keep only the non-updated elements to improve efficiency, reset index due to slice ###
    # productsDF = productsDF[productsDF.loc[:, (config.PRODUCT_ATTRIBUTES + config.DEEPFASHIONATTRIBUTESID)].isnull().apply(lambda x: all(x), axis=1)]
    productsDF = productsDF[productsDF.loc[:, config.PRODUCT_ATTRIBUTES].isnull().apply(lambda x: all(x), axis=1)]
    productsDF = productsDF.reset_index(drop=True)
    # productsDF = productsDF.drop(config.PRODUCT_ATTRIBUTES + config.DEEPFASHIONATTRIBUTESID, axis = 1)
    labelsDF['Oid'] = productsDF['Oid'].copy()
    ### Metadata and Headline consists of information related to each row ###

    metadata = productsDF['Metadata'].str.upper()
    headline = productsDF['Description'].str.upper()

    ### Read possible labels###

    labelsNRG = pd.read_excel(config.PRODUCT_ATTRIBUTES_PATH, sheet_name=config.SHEETNAME)
    
    # Create Variables with same name as the Energiers column names, to store the labels. Create new columns at assos with Energiers column names #
    attrDict = {}
    for attr in config.PRODUCT_ATTRIBUTES:
        attrDict[str(attr)] = list(labelsNRG[attr].replace(' ', np.nan).dropna().unique())
        attrDict[str(attr)] = [la.upper() for la in attrDict[str(attr)]]
        labelsDF[attr] = np.empty((len(productsDF), 0)).tolist()
    
    ### Preprocessing ###
    print('Preprocessing metadata...')    
    # Convert every label, metadata and headline to uppercase #
    splitted_metadata = [s.split() if isinstance(s,str) else " " for s in metadata]
    splitted_headline = [s.split() if isinstance(s,str) else " " for s in headline]    
    
    ### Search for occurences of labels in metadata and headline. For category length if the next word is not a kind of cat (Trousers etc) then it is propably wrong so we get rid of it. ###
    cat = 'ProductCategory'
    # for attr in (config.PRODUCT_ATTRIBUTES + config.DEEPFASHIONATTRIBUTES):
    for attr in config.PRODUCT_ATTRIBUTES:
        saved_meta = [(index, label, s.find(label), 1) for label in (attrDict[str(attr)]) for index,s in enumerate(metadata) if contains_word(s, label)]
        saved_head = [(index, label, s.find(label), 0) for label in (attrDict[str(attr)]) for index,s in enumerate(headline) if contains_word(s, label)]
        for s in (saved_meta + saved_head):
            s = editStrings(s)
            labelsDF.loc[s[0],attr].append((s[1], s[2], s[3]))
            if attr == 'Length':
                flag = 0
                for i,strSen in enumerate(splitted_metadata+splitted_headline):
                    for j,sen in enumerate(strSen[:-1]):
                        if sen == s[1] and 'SLEEVE' == strSen[j+1] and flag == 0:
                            flag = 1             
                if flag == 1:
                    labelsDF.loc[s[0], attr].remove((s[1], s[2], s[3]))
    
    ### Find similar words, for example -> rounded and round and one of them is discarded ###
    # for attr in (config.PRODUCT_ATTRIBUTES + config.DEEPFASHIONATTRIBUTES):
    for attr in config.PRODUCT_ATTRIBUTES:
        # Τhe next line sorts the elements of the respective columns based on position and on metadata or headline (headline first and then position on each string)
        labelsDF.loc[:, attr] = pd.Series([sorted(list(ele), key = lambda tup: (tup[2], tup[1])) for ele in labelsDF.loc[:, attr]])
        labelsDF.loc[:, attr] = pd.Series([list(map(lambda x: x[0], ele)) for ele in labelsDF.loc[:, attr]])
        saved = defaultdict(list)
        for i, element in enumerate(labelsDF.loc[:, attr]):
            if len(element) >= 2:
                for (k, l1),(ki, l2) in itertools.combinations(enumerate(element), 2): 
                    if similar(l1[0], l2[0]) >= 0.8:
                        saved[i].append((ki,l2))

        for key,value in saved.items():
            uni = np.unique(value, axis = 0)
            for index, x in uni:
                # We reverse because remove always pop outs the first element while we want the last
                labelsDF.loc[key, attr].reverse()
                labelsDF.loc[key, attr].remove(x)
                labelsDF.loc[key, attr].reverse()

    ### Check if list is empty and in this case make it None ###
    # for attr in (config.PRODUCT_ATTRIBUTES + config.DEEPFASHIONATTRIBUTES):
    for attr in config.PRODUCT_ATTRIBUTES:
        labelsDF.loc[:, attr] = labelsDF[attr].apply(lambda x: ','.join(x) if x else None)
    #Extract unique labels from metadata and headline
    labelsUnique = {attr:set([l for label in labelsDF[attr].unique() if label for l in label.split(',')]) for attr in labelsDF.loc[:, config.PRODUCT_ATTRIBUTES].columns}

    # Read from database the labels 
    # for name in (config.PRODUCT_ATTRIBUTES + config.DEEPFASHIONATTRIBUTES):
    dfDict = {}
    for attr in config.PRODUCT_ATTRIBUTES:
        # dfDict[str(attr)+'_DB'] = pd.read_sql_query("SELECT * FROM %s.dbo.%s" % (dbName, attr), engine)
        dfDict[str(attr)+'_DB'] = pd.read_sql_query(' SELECT * FROM public.\"%s\"' % str(attr), engine)

    # Update the DB label tables with the new attributes    
    for (label, values) in labelsUnique.items():
        for v in values:
            if v not in dfDict[label + '_DB']['Description'].values:
                if label=='ProductSubcategory':
                    submitdf = pd.DataFrame([{'Description': v, 'AlternativeDescription': '', 'ProductCategory': None, 
                                            'Active': True, 'OptimisticLockField': None}])
                    # submitdf.to_sql(label, schema='%s.dbo' % dbName, con=engine, if_exists='append', index=False)
                    submitdf.to_sql(label, con=engine, if_exists='append', index=False)
                else:
                    submitdf = pd.DataFrame([{'Description': v, 'AlternativeDescription': '', 'Active': True,
                                            'OptimisticLockField': None}])
                    # submitdf.to_sql(label, schema='%s.dbo' % dbName, con=engine, if_exists='append', index=False)
                    submitdf.to_sql(label, con=engine, if_exists='append', index=False)
        
    print('Update product attributes')  
    ## Update Product table with the foreign key values of the updated attributes
    # re-load from database the updated attribute tables and create a dataframe for each 
    dfDict = {}
    for attr in config.PRODUCT_ATTRIBUTES:
        # dfDict[str(attr)+'_DB'] = pd.read_sql_query(''' SELECT * FROM %s.dbo.%s''' % (dbName, str(attr)), engine)
        dfDict[str(attr)+'_DB'] = pd.read_sql_query(''' SELECT * FROM public.\"%s\"''' % str(attr), engine)

    # Update Products table for each attribute
    for attr in config.PRODUCT_ATTRIBUTES:
        # If there are multiple attributes, select the first
        labelsDF.loc[labelsDF[attr].notnull(), attr] = labelsDF[labelsDF[attr].notnull()][attr].apply(lambda x: x.split(',')[0] if x else None)
        # Merge the updated attribute values to Product table
        mergedDF = labelsDF.merge(dfDict[str(attr)+'_DB'], left_on=attr, right_on='Description')[['Oid_x', 'Oid_y']]
        productsDF.loc[productsDF['Oid'].isin(mergedDF['Oid_x'].values), attr] = mergedDF['Oid_y'].values
    # Execute Product update query
    # productsDF.to_sql("temp_table", schema='%s.dbo' % dbName, con=engine, if_exists='replace', index=False)
    productsDF.to_sql("temp_table", con = engine, if_exists = 'replace', index = False)
    with engine.begin() as conn:
        conn.execute(config.UPDATESQLQUERY)

    print("--- %s seconds ---" % (time.time() - start_time))



Loading Product table...
Preprocessing metadata...
Update product attributes
--- 1.8553619384765625 seconds ---


In [60]:
labelsNRG

,ProductCategory,ProductSubcategory,Gender,LifeStage,TrendTheme,InspirationBackground,Length,Sleeve,CollarDesign,NeckDesign,Fit,SampleManufacturer,ProductionManufacturer,Colors
0,SET,SHORT SET,Man-Boy,Infant,BASIC LINE,CHILDRENSALON,SHORT,SHORT SLEEVE,POLO COLLAR,ROUND NECK,REGULAR FIT,APPLE APPARELSS,APPLE APPARELSS,"ΓΚΡΙ_175,175,175"
1,BERMUDAS-SHORTS,SHORTS,Woman-Girl,0 to 5,BEACH & SPORT,PINTEREST,LONG,LONG SLEEVE,SHIRT COLLAR,COLLAR,CARGO,NaN,NaN,ΕΜΠΡΙΜΕ_
2,BLOUSES,BLOUSE POLO SHORT SLEEVE,,1 to 5,BEACH AND SPORT,STYLE RIGHT,MEDIUM,TURN UP SLEEVE,FLAT KNITTED RIB,TURTLENECK,RELAXED FIT,JESSE GARMENTS LTD,JESSE GARMENTS LTD,"ΛΕΥΚΟ_255,255,255"
3,TROUSERS,JEANS,,1 to 16,COOL GUYS,SHUTTERSTOCK,KNEE,SLEEVELESS,MAO COLLAR,HOODED,SLIM FIT,BEBESAN TEKS.SAV.VE DIS TIC.LTD.STI.,BEBESAN TEKS.SAV.VE DIS TIC.LTD.STI.,"ΜΠΛΕ ΤΖΗΝ_6,61,121"
4,ROMPER,INFANT ROMPER,,6 to 16,FREE LIFE,PITTI IMAGINE,CAPRI,RAGLAN SLEEVE,STAND UP COLLAR,V NECK,CHINOS,XIAMEN MICROUNION IND.,MINIDUNYA TEKSTIL HAKNUR BEBE,"ΜΑΡΕΝ_021,021,053"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ΜΠΕΖ_189,174,136,ΣΚΟΥΡΟ ΜΠΛΕ_046,095,255"
331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ΚΟΚΚΙΝΟ_242,011,046,ΜΑΡΕΝ_021,021,053,ΡΟΖ_255,..."
332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ΡΟΖ_255,128,192"
333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ΜΠΛΕ_28,28,255,ΣΟΜΟΝ_255,166,166"


In [27]:
mergedDF = labelsDF.merge(dfDict[str(attr)+'_DB'], left_on=attr, right_on='Description')[['Oid_x', 'Oid_y']]
filled_positions = productsDF['Oid'].isin(mergedDF['Oid_x'].values)
productsDF.loc[filled_positions, attr] = mergedDF['Oid_y'].values

In [47]:
mergedDF.merge(productsDF.loc[filled_positions], left_on='Oid_x', right_on='Oid')[attr].sum()

0

In [54]:
productsDF['Oid'].isin(mergedDF['Oid_x'].values)

0      False
1      False
2      False
3      False
4      False
       ...  
139     True
140    False
141    False
142     True
143     True
Name: Oid, Length: 144, dtype: bool

In [55]:
dfDict[str(attr)+'_DB']

,Oid,CreatedBy,UpdatedBy,CreatedOn,UpdatedOn,Description,AlternativeDescription,Active,Ordering,OptimisticLockField
0,10,None,None,2020-11-03 13:00:48,2020-11-03 13:00:48,SLIM FIT,,True,None,None
1,11,None,None,2020-11-03 13:00:48,2020-11-03 13:00:48,CARGO,,True,None,None
2,12,None,None,2020-11-03 13:00:48,2020-11-03 13:00:48,RELAXED FIT,,True,None,None
3,13,None,None,2020-11-03 13:00:48,2020-11-03 13:00:48,REGULAR FIT,,True,None,None
4,1,,,2020-10-17 11:55:00,2020-10-17 11:55:00,REGULAR FIT,,True,,
5,2,,,2020-10-17 11:55:00,2020-10-17 11:55:00,CARGO,,True,,
6,3,,,2020-10-17 11:55:00,2020-10-17 11:55:00,RELAXED FIT,,True,,
7,4,,,2020-10-17 11:55:00,2020-10-17 11:55:00,SLIM FIT,,True,,
8,5,,,2020-10-17 11:55:00,2020-10-17 11:55:00,CHINOS,,True,,
9,6,,,2020-10-17 11:55:00,2020-10-17 11:55:00,BIKER,,True,,


In [23]:
productsDF.loc[productsDF['Oid'].isin(mergedDF['Oid_x'].values), 'Oid']

7      6163
9      6167
10     6169
12     6108
14     6104
15     6105
16     6106
17     6107
18     6109
19     6110
20     6111
22     6112
27     6117
33     6124
34     6125
35     6126
36     6127
37     6128
51     6143
52     6158
53     6144
54     6145
58     6149
59     6151
61     6154
62     6156
63     6159
64     6161
67     6168
69     6171
79     6182
80     6183
81     6184
82     6185
83     6186
84     6187
85     6188
86     6189
87     6190
88     6191
111    6214
112    6215
113    6216
114    6217
115    6218
117    6219
118    6220
119    6221
132    6236
136    6240
137    6241
138    6242
139    6243
142    6246
143    6247
Name: Oid, dtype: int64

In [21]:
productsDF['Oid'].isin(mergedDF['Oid_x'].values)

0      False
1      False
2      False
3      False
4      False
       ...  
139     True
140    False
141    False
142     True
143     True
Name: Oid, Length: 144, dtype: bool

In [19]:
productsDF.loc[productsDF['Oid'].isin(mergedDF['Oid_x'].values), attr]

7      None
9      None
10     None
12     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
22     None
27     None
33     None
34     None
35     None
36     None
37     None
51     None
52     None
53     None
54     None
58     None
59     None
61     None
62     None
63     None
64     None
67     None
69     None
79     None
80     None
81     None
82     None
83     None
84     None
85     None
86     None
87     None
88     None
111    None
112    None
113    None
114    None
115    None
117    None
118    None
119    None
132    None
136    None
137    None
138    None
139    None
142    None
143    None
Name: Fit, dtype: object

In [16]:
productsDF.loc[productsDF['Oid'].isin(mergedDF['Oid_x'].values), attr]

7      None
9      None
10     None
12     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
22     None
27     None
33     None
34     None
35     None
36     None
37     None
51     None
52     None
53     None
54     None
58     None
59     None
61     None
62     None
63     None
64     None
67     None
69     None
79     None
80     None
81     None
82     None
83     None
84     None
85     None
86     None
87     None
88     None
111    None
112    None
113    None
114    None
115    None
117    None
118    None
119    None
132    None
136    None
137    None
138    None
139    None
142    None
143    None
Name: Fit, dtype: object

In [3]:
#DATABASE/ASSOS#
###Read Table Products from S4F DB###
cwd = helper_functions.CWD
engine = helper_functions.ENGINE
dbName = helper_functions.DB_NAME

# productsDF = pd.read_sql_query('SELECT * FROM S4F.dbo.Product', engine)
productsDF = pd.read_sql_query('SELECT * FROM public.\"Product\"', engine)

labelsDF = pd.DataFrame()

###Filter the old elements with the new one, Keep only the non-updated elements to improve efficiency, reset index due to slice###
# productsDF = productsDF[productsDF.loc[:, (constants.NRGATTRIBUTESID + constants.DEEPFASHIONATTRIBUTESID)].isnull().apply(lambda x: all(x), axis=1)]
productsDF = productsDF[productsDF.loc[:, constants.NRGATTRIBUTESID].isnull().apply(lambda x: all(x), axis=1)]
productsDF = productsDF.reset_index(drop=True)
#productsDF = productsDF.drop(constants.NRGATTRIBUTESID + constants.DEEPFASHIONATTRIBUTESID, axis = 1)
labelsDF['Oid'] = productsDF['Oid'].copy()
###Metadata and Headline consists of information related to each row###

metadata = productsDF['Metadata'].str.upper()
headline = productsDF['Description'].str.upper()
#NRG#

###Read possible labels from Energiers###

labelsNRG = pd.read_excel(constants.NRGATTRIBUTESPATH, sheet_name=constants.SHEETNAME)

###Create Variables with same name as the Energiers column names, to store the labels. Create new columns at assos with Energiers column names###
attrDict = {}
for attr in constants.NRGATTRIBUTES:
    attrDict[str(attr)] = labelsNRG[attr].replace(' ', np.nan).dropna().unique()
    labelsDF[attr] = np.empty((len(productsDF), 0)).tolist()

#Preprocessing#

###Convert every label, metadata and headline to uppercase###
# for label in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
for attr in constants.NRGATTRIBUTES:
    attrDict[str(attr)] = list(attrDict[str(attr)])
    attrDict[str(attr)] = [la.upper() for la in attrDict[str(attr)]]
#DES LIGO TO TRAINING STO PYTORCH ME TO PREPROCESSING GIA METADATA KAI HEADLINE SAN TRAIN KAI TEST

splitted_metadata = [s.split() if isinstance(s,str) else " " for s in metadata]
splitted_headline = [s.split() if isinstance(s,str) else " " for s in headline]


###Search for occurences of labels in metadata and headline. For category length if the next word is not a kind of cat (Trousers etc) then it is propably wrong so we get rid of it.###
cat = 'ProductCategory'
# for attr in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
for attr in constants.NRGATTRIBUTES:
    saved_meta = [(index, label, s.find(label), 1) for label in (attrDict[str(attr)]) for index,s in enumerate(metadata) if contains_word(s, label)]
    saved_head = [(index, label, s.find(label), 0) for label in (attrDict[str(attr)]) for index,s in enumerate(headline) if contains_word(s, label)]
    for s in (saved_meta + saved_head):
        s = editStrings(s)
        labelsDF.loc[s[0],attr].append((s[1], s[2], s[3]))
        if attr == 'Length':
            flag = 0
            for i,strSen in enumerate(splitted_metadata+splitted_headline):
                for j,sen in enumerate(strSen[:-1]):
                    if sen == s[1] and 'SLEEVE' == strSen[j+1] and flag == 0:
                        flag = 1             
            if flag == 1:
                #print(labelsDF.loc[s[0], attr])
                labelsDF.loc[s[0], attr].remove((s[1], s[2], s[3]))

###Find similar words, for example -> rounded and round and one of them is discarded###

# for attr in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
for attr in constants.NRGATTRIBUTES:
    #Τhe next line sorts the elements of the respective columns alphabetically
    #labelsDF.loc[:, col] = pd.Series([sorted(list(ele)) for ele in labelsDF.loc[:, col]])
    #Τhe next line sorts the elements of the respective columns based on position and on metadata or headline (headline first and then position on each string)
    labelsDF.loc[:, attr] = pd.Series([sorted(list(ele), key = lambda tup: (tup[2], tup[1])) for ele in labelsDF.loc[:, attr]])
    labelsDF.loc[:, attr] = pd.Series([list(map(lambda x: x[0], ele)) for ele in labelsDF.loc[:, attr]])
    saved = defaultdict(list)
    for i, element in enumerate(labelsDF.loc[:, attr]):
        if len(element) >= 2:
            for (k, l1),(ki, l2) in itertools.combinations(enumerate(element), 2): 
                if similar(l1[0], l2[0]) >= 0.8:
                    saved[i].append((ki,l2))

    for key,value in saved.items():
        uni = np.unique(value, axis = 0)
        for index, x in uni:
            #We reverse because remove always pop outs the first element while we want the last
            labelsDF.loc[key, attr].reverse()
            labelsDF.loc[key, attr].remove(x)
            labelsDF.loc[key, attr].reverse()

##Kane to sort se ola -> applymap
###Check if list is empty and in this case make it None###
# for attr in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
for attr in constants.NRGATTRIBUTES:
    labelsDF.loc[:, attr] = labelsDF[attr].apply(lambda x: ','.join(x) if x else None)
#Extract unique labels from metadata and headline
labelsUnique = {attr:set([l for label in labelsDF[attr].unique() if label for l in label.split(',')]) for attr in labelsDF.loc[:, constants.NRGATTRIBUTES].columns}
##MAYBE USE APPLYMAP for example
#labelsDF.applymap(lambda x: None if not x else ','.join(x))
#Upload to S4F DB#

#Read from database the labels 
# for name in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
dfDict = {}
for attr in constants.NRGATTRIBUTES:
    # dfDict[str(attr)+'_DB'] = pd.read_sql_query("SELECT * FROM %s.dbo.%s" % (DB_NAME, attr), engine)
    dfDict[str(attr)+'_DB'] = pd.read_sql_query(' SELECT * FROM public.\"%s\"' % str(attr), engine)

# Update the DB label tables with the new attributes    
for (label, values) in labelsUnique.items():
    for v in values:
        if v not in dfDict[label + '_DB']['Description'].values:
            if label=='ProductSubcategory':
                submitdf = pd.DataFrame([{'Description': v, 'AlternativeDescription': '', 'ProductCategory': None, 
                                        'Active': True, 'OptimisticLockField': None}])
                # submitdf.to_sql(label, schema='%s.dbo' % DB_NAME, con=engine, if_exists='append', index=False)
                submitdf.to_sql(label, con=engine, if_exists='append', index=False)
            else:
                submitdf = pd.DataFrame([{'Description': v, 'AlternativeDescription': '', 'Active': True,
                                        'OptimisticLockField': None}])
                # submitdf.to_sql(label, schema='%s.dbo' % DB_NAME, con=engine, if_exists='append', index=False)
                submitdf.to_sql(label, con=engine, if_exists='append', index=False)

## Update Product table with the foreign key values of the updated attributes
# re-load from database the updated attribute tables and create a dataframe for each 
dfDict = {}
for attr in constants.NRGATTRIBUTES:
    dfDict[str(attr)+'_DB'] = pd.read_sql_query(constants.SELECTQUERY + '\"%s\"' % str(attr), engine)

# Update Products table for each attribute
for attr in constants.NRGATTRIBUTES:
    # If there are multiple attributes, select the first
    labelsDF.loc[labelsDF[attr].notnull(), attr] = labelsDF[labelsDF[attr].notnull()][attr].apply(lambda x: x.split(',')[0] if x else None)
    # Merge the updated attribute values to Product table
    mergedDF = labelsDF.merge(dfDict[str(attr)+'_DB'], left_on=attr, right_on='Description')[['Oid_x', 'Oid_y']]
    productsDF.loc[productsDF['Oid'].isin(mergedDF['Oid_x'].values), attr] = mergedDF['Oid_y'].values
# Execute Product update query
updateQuery(productsDF, engine)


print("--- %s seconds ---" % (time.time() - start_time))

--- 9.220300912857056 seconds ---


In [9]:
labelsDF['Fit'].unique()

array([None], dtype=object)

In [10]:
productsDF

,Oid,CreatedBy,UpdatedBy,CreatedOn,UpdatedOn,Description,AlternativeDescription,Active,Ordering,ProductCode,...,LifeStage,TrendTheme,InspirationBackground,Gender,BusinessUnit,Season,Cluster,FinancialCluster,SumOfPercentage,OptimisticLockField
0,786,None,None,2020-10-30 18:10:59,2020-10-30 18:10:59,Activewear,None,true,0,None,...,None,None,None,NaN,None,None,None,None,None,None
1,880,None,None,2020-10-30 18:14:01,2020-10-30 18:14:01,Loungewear,None,true,0,None,...,None,None,None,NaN,None,None,None,None,None,None
2,929,None,None,2020-10-30 18:16:36,2020-10-30 18:16:36,Swimwear & Beachwear,None,true,0,None,...,None,None,None,NaN,None,None,None,None,None,None
3,927,None,None,2020-10-30 18:16:25,2020-10-30 18:16:25,Swimwear & Beachwear,None,true,0,None,...,None,None,None,NaN,None,None,None,None,None,None
4,831,None,None,2020-10-30 18:12:36,2020-10-30 18:12:36,Hoodies & Sweatshirts,None,true,0,None,...,None,None,None,NaN,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,1370,None,None,2020-11-03 11:19:29,2020-11-03 11:19:29,New in,None,true,0,1886280,...,None,None,None,2.0,None,None,None,None,None,None
136,1379,None,None,2020-11-03 11:19:40,2020-11-03 11:19:40,New in,None,true,0,None,...,None,None,None,NaN,None,None,None,None,None,None
137,1382,None,None,2020-11-03 11:19:43,2020-11-03 11:19:43,New in,None,true,0,None,...,None,None,None,NaN,None,None,None,None,None,None
138,1383,None,None,2020-11-03 11:19:43,2020-11-03 11:19:43,New in,None,true,0,None,...,None,None,None,NaN,None,None,None,None,None,None


In [3]:
#DATABASE/ASSOS#
###Read Table Products from S4F DB###
cwd = helper_functions.CWD
engine = helper_functions.ENGINE
dbName = helper_functions.DB_NAME

# productsDF = pd.read_sql_query('SELECT * FROM S4F.dbo.Product', engine)
productsDF = pd.read_sql_query('SELECT * FROM public.\"Product\"', engine)

labelsDF = pd.DataFrame()

###Filter the old elements with the new one, Keep only the non-updated elements to improve efficiency, reset index due to slice###
# productsDF = productsDF[productsDF.loc[:, (constants.NRGATTRIBUTESID + constants.DEEPFASHIONATTRIBUTESID)].isnull().apply(lambda x: all(x), axis=1)]
productsDF = productsDF[productsDF.loc[:, constants.NRGATTRIBUTESID].isnull().apply(lambda x: all(x), axis=1)]
productsDF = productsDF.reset_index(drop=True)
#productsDF = productsDF.drop(constants.NRGATTRIBUTESID + constants.DEEPFASHIONATTRIBUTESID, axis = 1)
labelsDF['Oid'] = productsDF['Oid'].copy()
###Metadata and Headline consists of information related to each row###

metadata = productsDF['Metadata'].str.upper()
headline = productsDF['Description'].str.upper()
#NRG#

###Read possible labels from Energiers###

labelsNRG = pd.read_excel(constants.NRGATTRIBUTESPATH, sheet_name=constants.SHEETNAME)

###Create Variables with same name as the Energiers column names, to store the labels. Create new columns at assos with Energiers column names###
attrDict = {}
for attr in constants.NRGATTRIBUTES:
    attrDict[str(attr)] = labelsNRG[attr].replace(' ', np.nan).dropna().unique()
    labelsDF[attr] = np.empty((len(productsDF), 0)).tolist()

#Preprocessing#

###Convert every label, metadata and headline to uppercase###
# for label in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
for attr in constants.NRGATTRIBUTES:
    attrDict[str(attr)] = list(attrDict[str(attr)])
    attrDict[str(attr)] = [la.upper() for la in attrDict[str(attr)]]
#DES LIGO TO TRAINING STO PYTORCH ME TO PREPROCESSING GIA METADATA KAI HEADLINE SAN TRAIN KAI TEST

splitted_metadata = [s.split() if isinstance(s,str) else " " for s in metadata]
splitted_headline = [s.split() if isinstance(s,str) else " " for s in headline]


###Search for occurences of labels in metadata and headline. For category length if the next word is not a kind of cat (Trousers etc) then it is propably wrong so we get rid of it.###
cat = 'ProductCategory'
# for attr in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
for attr in constants.NRGATTRIBUTES:
    saved_meta = [(index, label, s.find(label), 1) for label in (attrDict[str(attr)]) for index,s in enumerate(metadata) if contains_word(s, label)]
    saved_head = [(index, label, s.find(label), 0) for label in (attrDict[str(attr)]) for index,s in enumerate(headline) if contains_word(s, label)]
    for s in (saved_meta + saved_head):
        s = editStrings(s)
        labelsDF.loc[s[0],attr].append((s[1], s[2], s[3]))
        if attr == 'Length':
            flag = 0
            for i,strSen in enumerate(splitted_metadata+splitted_headline):
                for j,sen in enumerate(strSen[:-1]):
                    if sen == s[1] and 'SLEEVE' == strSen[j+1] and flag == 0:
                        flag = 1             
            if flag == 1:
                #print(labelsDF.loc[s[0], attr])
                labelsDF.loc[s[0], attr].remove((s[1], s[2], s[3]))

###Find similar words, for example -> rounded and round and one of them is discarded###

# for attr in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
for attr in constants.NRGATTRIBUTES:
    #Τhe next line sorts the elements of the respective columns alphabetically
    #labelsDF.loc[:, col] = pd.Series([sorted(list(ele)) for ele in labelsDF.loc[:, col]])
    #Τhe next line sorts the elements of the respective columns based on position and on metadata or headline (headline first and then position on each string)
    labelsDF.loc[:, attr] = pd.Series([sorted(list(ele), key = lambda tup: (tup[2], tup[1])) for ele in labelsDF.loc[:, attr]])
    labelsDF.loc[:, attr] = pd.Series([list(map(lambda x: x[0], ele)) for ele in labelsDF.loc[:, attr]])
    saved = defaultdict(list)
    for i, element in enumerate(labelsDF.loc[:, attr]):
        if len(element) >= 2:
            for (k, l1),(ki, l2) in itertools.combinations(enumerate(element), 2): 
                if similar(l1[0], l2[0]) >= 0.8:
                    saved[i].append((ki,l2))

    for key,value in saved.items():
        uni = np.unique(value, axis = 0)
        for index, x in uni:
            #We reverse because remove always pop outs the first element while we want the last
            labelsDF.loc[key, attr].reverse()
            labelsDF.loc[key, attr].remove(x)
            labelsDF.loc[key, attr].reverse()

##Kane to sort se ola -> applymap
###Check if list is empty and in this case make it None###
# for attr in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
for attr in constants.NRGATTRIBUTES:
    labelsDF.loc[:, attr] = labelsDF[attr].apply(lambda x: ','.join(x) if x else None)
#Extract unique labels from metadata and headline
labelsUnique = {attr:set([l for label in labelsDF[attr].unique() if label for l in label.split(',')]) for attr in labelsDF.loc[:, constants.NRGATTRIBUTES].columns}
##MAYBE USE APPLYMAP for example
#labelsDF.applymap(lambda x: None if not x else ','.join(x))
#Upload to S4F DB#

#Read from database the labels 
# for name in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
dfDict = {}
for attr in constants.NRGATTRIBUTES:
    # dfDict[str(attr)+'_DB'] = pd.read_sql_query("SELECT * FROM %s.dbo.%s" % (DB_NAME, attr), engine)
    dfDict[str(attr)+'_DB'] = pd.read_sql_query(' SELECT * FROM public.\"%s\"' % str(attr), engine)

# Update the DB label tables with the new attributes    
for (label, values) in labelsUnique.items():
    for v in values:
        if v not in dfDict[label + '_DB']['Description'].values:
            if label=='ProductSubcategory':
                submitdf = pd.DataFrame([{'Description': v, 'AlternativeDescription': '', 'ProductCategory': None, 
                                        'Active': True, 'OptimisticLockField': None}])
                # submitdf.to_sql(label, schema='%s.dbo' % DB_NAME, con=engine, if_exists='append', index=False)
                submitdf.to_sql(label, con=engine, if_exists='append', index=False)
            else:
                submitdf = pd.DataFrame([{'Description': v, 'AlternativeDescription': '', 'Active': True,
                                        'OptimisticLockField': None}])
                # submitdf.to_sql(label, schema='%s.dbo' % DB_NAME, con=engine, if_exists='append', index=False)
                submitdf.to_sql(label, con=engine, if_exists='append', index=False)

## Update Product table with the foreign key values of the updated attributes
# re-load from database the updated attribute tables and create a dataframe for each 
dfDict = {}
for attr in constants.NRGATTRIBUTES:
    dfDict[str(attr)+'_DB'] = pd.read_sql_query(constants.SELECTQUERY + '\"%s\"' % str(attr), engine)

# Update Products table for each attribute
for attr in constants.NRGATTRIBUTES:
    # If there are multiple attributes, select the first
    labelsDF.loc[labelsDF[attr].notnull(), attr] = labelsDF[labelsDF[attr].notnull()][attr].apply(lambda x: x.split(',')[0] if x else None)
    # Merge the updated attribute values to Product table
    mergedDF = labelsDF.merge(dfDict[str(attr)+'_DB'], left_on=attr, right_on='Description')[['Oid_x', 'Oid_y']]
    productsDF.loc[productsDF['Oid'].isin(mergedDF['Oid_x'].values), attr] = mergedDF['Oid_y'].values

# updatedf.to_sql("Product", schema='%s.dbo' % DB_NAME, con=engine, if_exists='replace', index=False)
#     productsDF.to_sql("Product", con=engine, if_exists='replace', index=False)


#     print("--- %s seconds ---" % (time.time() - start_time))

















In [16]:
productsDF.to_sql("temp_table", con = engine, if_exists = 'replace', index = False)

In [ ]:
, public.\"Product\".\"ProductSubcategory\" = temp_table.\"ProductSubcategory\", public.\"Product\".\"Length\" = temp_table.\"Length\", public.\"Product\".\"Sleeve\" = temp_table.\"Sleeve\", public.\"Product\".\"CollarDesign\" = temp_table.\"CollarDesign\", public.\"Product\".\"NeckDesign\" = temp_table.\"NeckDesign\", public.\"Product\".\"Fit\" = temp_table.\"Fit\"

In [21]:
UPDATESQLQUERY = """
    UPDATE "Product" 
    SET "ProductCategory" = temp_table."ProductCategory", "ProductSubcategory" = temp_table."ProductSubcategory", "Length" = temp_table."Length", "Sleeve" = temp_table."Sleeve", "CollarDesign" = temp_table."CollarDesign", "NeckDesign" = temp_table."NeckDesign", "Fit" = temp_table."Fit"
    FROM temp_table 
    WHERE public."Product"."Oid" = public."temp_table"."Oid"
"""

In [22]:
updateQuery(productsDF, engine)

In [8]:
constants.UPDATESQLQUERY

'\n    UPDATE public."Product"\n    SET public."Product".ProductCategory = temp_table.ProductCategory, public."Product".ProductSubcategory = temp_table.ProductSubcategory, public."Product".Length = temp_table.Length, public."Product".Sleeve = temp_table.Sleeve, public."Product".CollarDesign = temp_table.CollarDesign, public."Product".NeckDesign = temp_table.NeckDesign, public."Product".Fit = temp_table.Fit\n    FROM public."Product"\n    INNER JOIN temp_table\n    ON (public."Product".Oid = temp_table.Oid)\n    WHERE public."Product".Oid = temp_table.Oid\n'

In [3]:
#DATABASE/ASSOS#
###Read Table Products from S4F DB###
cwd = helper_functions.CWD
engine = helper_functions.ENGINE
dbName = helper_functions.DB_NAME

# query_db = pd.read_sql_query('SELECT * FROM S4F.dbo.Product', engine)
query_db = pd.read_sql_query('SELECT * FROM public.\"Product\"', engine)

assosDF = pd.DataFrame(query_db)
labelsDF = pd.DataFrame()

###Filter the old elements with the new one, Keep only the non-updated elements to improve efficiency, reset index due to slice###
# assosDF = assosDF[assosDF.loc[:, (constants.NRGATTRIBUTESID + constants.DEEPFASHIONATTRIBUTESID)].isnull().apply(lambda x: all(x), axis=1)]
assosDF = assosDF[assosDF.loc[:, constants.NRGATTRIBUTESID].isnull().apply(lambda x: all(x), axis=1)]
assosDF = assosDF.reset_index(drop=True)
#print(assosDF.columns)
#assosDF = assosDF.drop(constants.NRGATTRIBUTESID + constants.DEEPFASHIONATTRIBUTESID, axis = 1)
labelsDF['Oid'] = assosDF['Oid'].copy()
###Metadata and Headline consists of information related to each row###

metadata = assosDF['Metadata'].str.upper()
headline = assosDF['Description'].str.upper()
#NRG#

###Read possible labels from Energiers###

labelsNRG = pd.read_excel(constants.NRGATTRIBUTESPATH, sheet_name=constants.SHEETNAME)

###Create Variables with same name as the Energiers column names, to store the labels. Create new columns at assos with Energiers column names###

for at in constants.NRGATTRIBUTES:
    locals()[str(at)] = labelsNRG[at].replace(' ', np.nan).dropna().unique()
    labelsDF[at] = np.empty((len(assosDF), 0)).tolist()
#DEEPFASHION#

# groups = preprocessDeepfashion()
# for attribute in constants.DEEPFASHIONATTRIBUTES:
#     locals()[str(attribute)] = groups.get_group(str(attribute[0]) + str(attribute[1:]))
#     labelsDF[attribute] = np.empty((len(assosDF), 0)).tolist()

#Preprocessing#

###Convert every label, metadata and headline to uppercase###
# for label in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
for label in (constants.NRGATTRIBUTES):
    locals()[str(label)] = list(locals()[str(label)])
    locals()[str(label)] = [la.upper() for la in locals()[str(label)]]
#DES LIGO TO TRAINING STO PYTORCH ME TO PREPROCESSING GIA METADATA KAI HEADLINE SAN TRAIN KAI TEST

splitted_metadata = [s.split() if isinstance(s,str) else " " for s in metadata]
splitted_headline = [s.split() if isinstance(s,str) else " " for s in headline]


###Search for occurences of labels in metadata and headline. For category length if the next word is not a kind of cat (Trousers etc) then it is propably wrong so we get rid of it.###
cat = 'ProductCategory'
# for col in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
for col in (constants.NRGATTRIBUTES):
    saved_meta = [(index, label, s.find(label), 1) for label in (locals()[str(col)]) for index,s in enumerate(metadata) if contains_word(s, label)]
    saved_head = [(index, label, s.find(label), 0) for label in (locals()[str(col)]) for index,s in enumerate(headline) if contains_word(s, label)]
    for s in (saved_meta + saved_head):
        s = editStrings(s)
        labelsDF.loc[s[0],col].append((s[1], s[2], s[3]))
        if col == 'Length':
            flag = 0
            for i,strSen in enumerate(splitted_metadata+splitted_headline):
                for j,sen in enumerate(strSen[:-1]):
                    if sen == s[1] and 'SLEEVE' == strSen[j+1] and flag == 0:
                        flag = 1             
            if flag == 1:
                #print(labelsDF.loc[s[0], col])
                labelsDF.loc[s[0], col].remove((s[1], s[2], s[3]))

###Find similar words, for example -> rounded and round and one of them is discarded###

# for col in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
for col in (constants.NRGATTRIBUTES):
    #Τhe next line sorts the elements of the respective columns alphabetically
    #labelsDF.loc[:, col] = pd.Series([sorted(list(ele)) for ele in labelsDF.loc[:, col]])
    #Τhe next line sorts the elements of the respective columns based on position and on metadata or headline (headline first and then position on each string)
    labelsDF.loc[:, col] = pd.Series([sorted(list(ele), key = lambda tup: (tup[2], tup[1])) for ele in labelsDF.loc[:, col]])
    labelsDF.loc[:, col] = pd.Series([list(map(lambda x: x[0], ele)) for ele in labelsDF.loc[:, col]])
    saved = defaultdict(list)
    for i, element in enumerate(labelsDF.loc[:, col]):
        if len(element) >= 2:
            for (k, l1),(ki, l2) in itertools.combinations(enumerate(element), 2): 
                if similar(l1[0], l2[0]) >= 0.8:
                    saved[i].append((ki,l2))

    for key,value in saved.items():
        uni = np.unique(value, axis = 0)
        for index, x in uni:
            #We reverse because remove always pop outs the first element while we want the last
            labelsDF.loc[key, col].reverse()
            labelsDF.loc[key, col].remove(x)
            labelsDF.loc[key, col].reverse()

for col in (constants.NRGATTRIBUTES):
    labelsDF.loc[:, col] = labelsDF[col].apply(lambda x: ','.join(x) if x else None)
#Extract unique labels from metadata and headline
labelsUnique = {col:set([l for label in labelsDF[col].unique() if label for l in label.split(',')]) for col in labelsDF.loc[:, constants.NRGATTRIBUTES].columns}

# Load from database the attribute tables and create a dataframe for each 
dfDict = {}
for name in (constants.NRGATTRIBUTES):
    dfDict[str(name)+'_DB'] = pd.read_sql_query(constants.SELECTQUERY + '\"%s\"' % str(name), engine)
    
# Update the DB label tables with the new attributes    
for (label, values) in labelsUnique.items():
    for v in values:
        if v not in dfDict[label + '_DB']['Description'].values:
            if label=='ProductSubcategory':
                submitdf = pd.DataFrame([{'Description': v, 'AlternativeDescription': '', 'ProductCategory': None, 
                                          'Active': True, 'OptimisticLockField': None}])
                # submitdf.to_sql(label, schema='%s.dbo' % DB_NAME, con=engine, if_exists='append', index=False)
                submitdf.to_sql(label, con=engine, if_exists='append', index=False)
            else:
                submitdf = pd.DataFrame([{'Description': v, 'AlternativeDescription': '', 'Active': True,
                                          'OptimisticLockField': None}])
                # submitdf.to_sql(label, schema='%s.dbo' % DB_NAME, con=engine, if_exists='append', index=False)
                submitdf.to_sql(label, con=engine, if_exists='append', index=False)
    
## Update Product table with the foreign key values of the updated attributes
# re-load from database the updated attribute tables and create a dataframe for each 
dfDict = {}
for name in (constants.NRGATTRIBUTES):
    dfDict[str(name)+'_DB'] = pd.read_sql_query(constants.SELECTQUERY + '\"%s\"' % str(name), engine)

# Update Products table for each attribute
for name in (constants.NRGATTRIBUTES):
    # If there are multiple attributes, select the first
    labelsDF.loc[labelsDF[name].notnull(), name] = labelsDF[labelsDF[name].notnull()][name].apply(lambda x: x.split(',')[0] if x else None)
    # Merge the updated attribute values to Product table
    mergedDF = labelsDF.merge(dfDict[str(name)+'_DB'], left_on=name, right_on='Description')[['Oid_x', 'Oid_y']]
    assosDF.loc[assosDF['Oid'].isin(mergedDF['Oid_x'].values), name] = mergedDF['Oid_y'].values

ValueError: Must have equal len keys and value when setting with an iterable

In [4]:
mergedDF

,Oid_x,Oid_y
0,NaN,32
1,NaN,32
2,NaN,32
3,NaN,32
4,NaN,32
5,NaN,32
6,NaN,32
7,NaN,32
8,NaN,32
9,NaN,32


In [134]:
for name in (constants.NRGATTRIBUTES):
    print(name, assosDF[name].values)

ProductCategory [None None None None None None None None None None None None None None
 None None None None None None 27 27 None 27 27 None None None None None
 None None None None None None None None None None 27 None 27 None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None 27 None None None None None 27 None None 27 27 22 None 22 25 25 None
 None None None None None None None None None None None None None None
 None None None 25 None None None None None None None None None 25 None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None 23 23 23 23 None 23 23 23 23 None None
 None None None None None None None None None None None None None None
 None None None None None None None None None Non

In [99]:
## Update Product table with the foreign key values of the updated attributes
# re-load from database the updated attribute tables and create a dataframe for each 
dfDict = {}
for name in (constants.NRGATTRIBUTES):
    dfDict[str(name)+'_DB'] = pd.read_sql_query(constants.SELECTQUERY + '\"%s\"' % str(name), engine)

# Update Products table for each attribute
for name in (constants.NRGATTRIBUTES):
    # If there are multiple attributes, select the first
    labelsDF.loc[labelsDF[name].notnull(), name] = labelsDF[labelsDF[name].notnull()][name].apply(lambda x: x.split(',')[0] if x else None)
    # Merge the updated attribute values to Product table
    mergedDF = labelsDF.merge(dfDict[str(name)+'_DB'], left_on=name, right_on='Description')[['Oid_x', 'Oid_y']]
    assosDF.loc[assosDF['Oid'].isin(mergedDF['Oid_x'].values), name] = mergedDF['Oid_y'].values

{'ProductCategory_DB':    Oid CreatedBy UpdatedBy           CreatedOn           UpdatedOn  \
 0   22      None      None 2020-11-02 20:54:15 2020-11-02 20:54:15   
 1   23      None      None 2020-11-02 20:54:15 2020-11-02 20:54:15   
 2   24      None      None 2020-11-02 20:54:15 2020-11-02 20:54:15   
 3   25      None      None 2020-11-02 20:54:15 2020-11-02 20:54:15   
 4   26      None      None 2020-11-02 20:54:15 2020-11-02 20:54:15   
 5   27      None      None 2020-11-02 20:54:15 2020-11-02 20:54:15   
 6   28      None      None 2020-11-02 20:54:15 2020-11-02 20:54:15   
 
   Description AlternativeDescription  Active Ordering OptimisticLockField  
 0       SKIRT                           True     None                None  
 1    TROUSERS                           True     None                None  
 2   TRACKSUIT                           True     None                None  
 3       DRESS                           True     None                None  
 4    LEGGINGS         

In [127]:
for name in (constants.NRGATTRIBUTES):
    # If there are multiple attributes, select the first
    labelsDF.loc[labelsDF[name].notnull(), name] = labelsDF[labelsDF[name].notnull()][name].apply(lambda x: x.split(',')[0] if x else None)
    # Merge the updated attribute values to Product table
    mergedDF = labelsDF.merge(dfDict[str(name)+'_DB'], left_on=name, right_on='Description')[['Oid_x', 'Oid_y']]
    assosDF.loc[assosDF['Oid'].isin(mergedDF['Oid_x'].values), name] = mergedDF['Oid_y'].values

assosDF

In [130]:
assosDF.loc[assosDF[name].notnull()]

,Oid,CreatedBy,UpdatedBy,CreatedOn,UpdatedOn,Description,AlternativeDescription,Active,Ordering,ProductCode,...,LifeStage,TrendTheme,InspirationBackground,Gender,BusinessUnit,Season,Cluster,FinancialCluster,SumOfPercentage,OptimisticLockField
0,774,None,None,2020-10-30 18:10:37.673144+02,2020-10-30 18:10:37.673144+02,New in,None,true,0,1803174,...,None,None,None,2.0,None,None,None,None,None,None
1,775,None,None,2020-10-30 18:10:39.034415+02,2020-10-30 18:10:39.034415+02,New in,None,true,0,1803195,...,None,None,None,2.0,None,None,None,None,None,None
2,776,None,None,2020-10-30 18:10:40.491406+02,2020-10-30 18:10:40.491406+02,New in,None,true,0,1803185,...,None,None,None,2.0,None,None,None,None,None,None
3,777,None,None,2020-10-30 18:10:41.885653+02,2020-10-30 18:10:41.885653+02,New in,None,true,0,1840074,...,None,None,None,2.0,None,None,None,None,None,None
5,779,None,None,2020-10-30 18:10:44.657992+02,2020-10-30 18:10:44.657992+02,New in,None,true,0,1771026,...,None,None,None,2.0,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,1114,None,None,2020-10-30 18:27:15.322747+02,2020-10-30 18:27:15.322747+02,Trousers & Chinos,None,true,0,1767155,...,None,None,None,1.0,None,None,None,None,None,None
341,1115,None,None,2020-10-30 18:27:18.330813+02,2020-10-30 18:27:18.330813+02,Trousers & Chinos,None,true,0,1783990,...,None,None,None,1.0,None,None,None,None,None,None
342,1116,None,None,2020-10-30 18:27:20.609484+02,2020-10-30 18:27:20.609484+02,Trousers & Chinos,None,true,0,1793349,...,None,None,None,1.0,None,None,None,None,None,None
343,1117,None,None,2020-10-30 18:27:23.654318+02,2020-10-30 18:27:23.654318+02,Trousers & Chinos,None,true,0,1793420,...,None,None,None,1.0,None,None,None,None,None,None


In [126]:
assosDF.loc[assosDF['Oid'].isin(mergedDF['Oid_x'].values), name] == mergedDF['Oid_y'].values

0      False
1      False
2      False
3      False
5      False
       ...  
340    False
341    False
342    False
343    False
344    False
Name: Fit, Length: 214, dtype: bool

In [111]:
assosDF.loc[assosDF['Oid'].isin(mergedDF['Oid_x'].values), name]

20     None
21     None
23     None
24     None
40     None
42     None
88     None
94     None
97     None
98     None
99     None
101    None
102    None
103    None
122    None
132    None
169    None
170    None
171    None
172    None
174    None
175    None
176    None
177    None
205    None
278    None
279    None
280    None
281    None
282    None
283    None
284    None
286    None
287    None
288    None
289    None
290    None
291    None
292    None
293    None
294    None
295    None
296    None
297    None
336    None
337    None
338    None
339    None
340    None
341    None
342    None
343    None
344    None
Name: ProductCategory, dtype: object

In [113]:
assosDF['Oid']

0       774
1       775
2       776
3       777
4       778
       ... 
347    1121
348    1122
349    1123
350    1124
351    1125
Name: Oid, Length: 352, dtype: int64

In [112]:
mergedDF['Oid_x']

0      794
1      795
2      798
3      868
4      869
5      873
6      874
7      875
8      893
9      978
10     797
11     903
12     814
13     816
14     864
15     867
16     859
17     943
18     944
19     945
20     946
21     947
22     948
23     949
24     950
25    1111
26    1112
27    1113
28    1114
29    1115
30    1116
31    1117
32    1118
33    1052
34    1053
35    1054
36    1055
37    1056
38    1057
39    1058
40    1059
41    1060
42    1061
43    1062
44    1063
45    1064
46    1065
47    1066
48    1067
49    1068
50    1069
51    1070
52    1110
Name: Oid_x, dtype: int64

In [106]:
name

'ProductCategory'

In [104]:
assosDF['Fit']

0      None
1      None
2      None
3      None
4      None
       ... 
347    None
348    None
349    None
350    None
351    None
Name: Fit, Length: 352, dtype: object

In [84]:
assosDF.loc[mergedDF['Oid_x'].index]

,Oid,CreatedBy,UpdatedBy,CreatedOn,UpdatedOn,Description,AlternativeDescription,Active,Ordering,ProductCode,...,LifeStage,TrendTheme,InspirationBackground,Gender,BusinessUnit,Season,Cluster,FinancialCluster,SumOfPercentage,OptimisticLockField
0,774,None,None,2020-10-30 18:10:37.673144+02,2020-10-30 18:10:37.673144+02,New in,None,true,0,1803174,...,None,None,None,2.0,None,None,None,None,None,None
1,775,None,None,2020-10-30 18:10:39.034415+02,2020-10-30 18:10:39.034415+02,New in,None,true,0,1803195,...,None,None,None,2.0,None,None,None,None,None,None
2,776,None,None,2020-10-30 18:10:40.491406+02,2020-10-30 18:10:40.491406+02,New in,None,true,0,1803185,...,None,None,None,2.0,None,None,None,None,None,None
3,777,None,None,2020-10-30 18:10:41.885653+02,2020-10-30 18:10:41.885653+02,New in,None,true,0,1840074,...,None,None,None,2.0,None,None,None,None,None,None
4,778,None,None,2020-10-30 18:10:43.351711+02,2020-10-30 18:10:43.351711+02,New in,None,true,0,1771012,...,None,None,None,2.0,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,982,None,None,2020-10-30 18:19:13.049467+02,2020-10-30 18:19:13.049467+02,Co-ords,None,true,0,None,...,None,None,None,NaN,None,None,None,None,None,None
210,983,None,None,2020-10-30 18:19:14.380213+02,2020-10-30 18:19:14.380213+02,Co-ords,None,true,0,None,...,None,None,None,NaN,None,None,None,None,None,None
211,984,None,None,2020-10-30 18:19:18.167448+02,2020-10-30 18:19:18.167448+02,Dungarees & Boiler Suits,None,true,0,1809664,...,None,None,None,1.0,None,None,None,None,None,None
212,985,None,None,2020-10-30 18:19:19.603471+02,2020-10-30 18:19:19.603471+02,Dungarees & Boiler Suits,None,true,0,1728759,...,None,None,None,1.0,None,None,None,None,None,None


In [90]:
mergedDF = labelsDF.merge(dfDict[str(name)+'_DB'], left_on=name, right_on='Description')[['Oid_x', 'Oid_y']]

mergedDF

,Oid_x,Oid_y
0,774,2
1,775,2
2,776,2
3,781,2
4,785,2
...,...,...
209,1114,4
210,1115,4
211,1116,4
212,1117,4


In [96]:
assosDF.loc[assosDF['Oid'].isin(mergedDF['Oid_x'].values), 'Fit'] == mergedDF['Oid_y']

0      None
1      None
2      None
3      None
5      None
       ... 
340    None
341    None
342    None
343    None
344    None
Name: Fit, Length: 214, dtype: object

In [87]:
for k,series in mergedDF.iterrows():
        assosDF.loc[assosDF['Oid']==series['Oid_x']].isin([3, 6])]

,Oid,CreatedBy,UpdatedBy,CreatedOn,UpdatedOn,Description,AlternativeDescription,Active,Ordering,ProductCode,...,InspirationBackground,Gender,BusinessUnit,Season,Cluster,FinancialCluster,SumOfPercentage,OptimisticLockField,Oid_x,Oid_y
0,774,None,None,2020-10-30 18:10:37.673144+02,2020-10-30 18:10:37.673144+02,New in,None,true,0,1803174,...,None,2.0,None,None,None,None,None,None,774,2
1,775,None,None,2020-10-30 18:10:39.034415+02,2020-10-30 18:10:39.034415+02,New in,None,true,0,1803195,...,None,2.0,None,None,None,None,None,None,775,2
2,776,None,None,2020-10-30 18:10:40.491406+02,2020-10-30 18:10:40.491406+02,New in,None,true,0,1803185,...,None,2.0,None,None,None,None,None,None,776,2
3,777,None,None,2020-10-30 18:10:41.885653+02,2020-10-30 18:10:41.885653+02,New in,None,true,0,1840074,...,None,2.0,None,None,None,None,None,None,777,1
4,779,None,None,2020-10-30 18:10:44.657992+02,2020-10-30 18:10:44.657992+02,New in,None,true,0,1771026,...,None,2.0,None,None,None,None,None,None,779,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,1114,None,None,2020-10-30 18:27:15.322747+02,2020-10-30 18:27:15.322747+02,Trousers & Chinos,None,true,0,1767155,...,None,1.0,None,None,None,None,None,None,1114,4
210,1115,None,None,2020-10-30 18:27:18.330813+02,2020-10-30 18:27:18.330813+02,Trousers & Chinos,None,true,0,1783990,...,None,1.0,None,None,None,None,None,None,1115,4
211,1116,None,None,2020-10-30 18:27:20.609484+02,2020-10-30 18:27:20.609484+02,Trousers & Chinos,None,true,0,1793349,...,None,1.0,None,None,None,None,None,None,1116,4
212,1117,None,None,2020-10-30 18:27:23.654318+02,2020-10-30 18:27:23.654318+02,Trousers & Chinos,None,true,0,1793420,...,None,1.0,None,None,None,None,None,None,1117,4


In [94]:
for k,series in mergedDF.iterrows():
    print(series['Oid_x'])

774
775
776
781
785
798
804
805
806
808
809
811
832
833
835
836
839
849
850
851
854
860
861
873
874
888
916
933
950
962
964
965
980
984
987
991
999
1029
1060
1061
777
779
784
790
791
792
796
797
799
802
807
810
814
816
817
818
834
838
840
844
845
847
848
852
853
855
857
858
859
862
863
865
866
868
870
875
876
877
895
897
900
901
902
903
904
905
915
917
919
920
921
940
942
944
946
1031
947
949
956
957
958
960
961
967
968
969
970
971
972
973
974
977
978
979
986
988
990
992
993
994
995
996
997
998
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1019
1024
1025
1026
1027
1028
1030
1033
1034
1035
1036
1039
1049
1041
1044
1045
1046
1052
1053
1054
1057
1058
1077
1059
1063
1064
1067
1068
1070
1071
1072
1073
1074
1075
1076
1078
1087
1088
1091
1092
1096
1107
1108
1109
794
795
819
821
841
842
867
869
918
932
953
989
1037
1062
1065
1066
1069
1089
963
985
1021
1111
1112
1113
1114
1115
1116
1117
1118


In [58]:
assosDF

,Oid,CreatedBy,UpdatedBy,CreatedOn,UpdatedOn,Description,AlternativeDescription,Active,Ordering,ProductCode,...,LifeStage,TrendTheme,InspirationBackground,Gender,BusinessUnit,Season,Cluster,FinancialCluster,SumOfPercentage,OptimisticLockField
0,774,None,None,2020-10-30 18:10:37.673144+02,2020-10-30 18:10:37.673144+02,New in,None,true,0,1803174,...,None,None,None,2.0,None,None,None,None,None,None
1,775,None,None,2020-10-30 18:10:39.034415+02,2020-10-30 18:10:39.034415+02,New in,None,true,0,1803195,...,None,None,None,2.0,None,None,None,None,None,None
2,776,None,None,2020-10-30 18:10:40.491406+02,2020-10-30 18:10:40.491406+02,New in,None,true,0,1803185,...,None,None,None,2.0,None,None,None,None,None,None
3,777,None,None,2020-10-30 18:10:41.885653+02,2020-10-30 18:10:41.885653+02,New in,None,true,0,1840074,...,None,None,None,2.0,None,None,None,None,None,None
4,778,None,None,2020-10-30 18:10:43.351711+02,2020-10-30 18:10:43.351711+02,New in,None,true,0,1771012,...,None,None,None,2.0,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,1121,None,None,2020-10-30 18:27:40.499653+02,2020-10-30 18:27:40.499653+02,Underwear,None,true,0,1795658,...,None,None,None,1.0,None,None,None,None,None,None
348,1122,None,None,2020-10-30 18:27:43.120599+02,2020-10-30 18:27:43.120599+02,Underwear,None,true,0,1801306,...,None,None,None,1.0,None,None,None,None,None,None
349,1123,None,None,2020-10-30 18:27:46.42055+02,2020-10-30 18:27:46.42055+02,Underwear,None,true,0,1811278,...,None,None,None,1.0,None,None,None,None,None,None
350,1124,None,None,2020-10-30 18:27:49.346475+02,2020-10-30 18:27:49.346475+02,Underwear,None,true,0,1824701,...,None,None,None,1.0,None,None,None,None,None,None


In [50]:
labelsDF.loc[labelsDF['Fit'].notnull(), 'Fit'] = labelsDF[labelsDF['Fit'].notnull()]['Fit'].apply(lambda x: x.split(',')[0] if x else None)

In [52]:
labelsDF

,Oid,ProductCategory,ProductSubcategory,Length,Sleeve,CollarDesign,NeckDesign,Fit
0,774,None,None,None,None,None,None,RELAXED FIT
1,775,None,None,None,None,None,None,RELAXED FIT
2,776,None,None,None,None,None,None,RELAXED FIT
3,777,None,None,None,None,None,None,REGULAR FIT
4,778,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
347,1121,None,None,None,None,None,None,None
348,1122,None,None,None,None,None,None,None
349,1123,None,None,None,None,None,None,None
350,1124,None,None,None,None,None,None,None


In [36]:
assosDF

,Oid,CreatedBy,UpdatedBy,CreatedOn,UpdatedOn,Description,AlternativeDescription,Active,Ordering,ProductCode,...,LifeStage,TrendTheme,InspirationBackground,Gender,BusinessUnit,Season,Cluster,FinancialCluster,SumOfPercentage,OptimisticLockField
0,774,None,None,2020-10-30 18:10:37.673144+02,2020-10-30 18:10:37.673144+02,New in,None,true,0,1803174,...,None,None,None,2.0,None,None,None,None,None,None
1,775,None,None,2020-10-30 18:10:39.034415+02,2020-10-30 18:10:39.034415+02,New in,None,true,0,1803195,...,None,None,None,2.0,None,None,None,None,None,None
2,776,None,None,2020-10-30 18:10:40.491406+02,2020-10-30 18:10:40.491406+02,New in,None,true,0,1803185,...,None,None,None,2.0,None,None,None,None,None,None
3,777,None,None,2020-10-30 18:10:41.885653+02,2020-10-30 18:10:41.885653+02,New in,None,true,0,1840074,...,None,None,None,2.0,None,None,None,None,None,None
4,778,None,None,2020-10-30 18:10:43.351711+02,2020-10-30 18:10:43.351711+02,New in,None,true,0,1771012,...,None,None,None,2.0,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,1121,None,None,2020-10-30 18:27:40.499653+02,2020-10-30 18:27:40.499653+02,Underwear,None,true,0,1795658,...,None,None,None,1.0,None,None,None,None,None,None
348,1122,None,None,2020-10-30 18:27:43.120599+02,2020-10-30 18:27:43.120599+02,Underwear,None,true,0,1801306,...,None,None,None,1.0,None,None,None,None,None,None
349,1123,None,None,2020-10-30 18:27:46.42055+02,2020-10-30 18:27:46.42055+02,Underwear,None,true,0,1811278,...,None,None,None,1.0,None,None,None,None,None,None
350,1124,None,None,2020-10-30 18:27:49.346475+02,2020-10-30 18:27:49.346475+02,Underwear,None,true,0,1824701,...,None,None,None,1.0,None,None,None,None,None,None


In [5]:
for (label, values) in labelsUnique.items():
    for v in values:
        if v not in dfDict[label + '_DB']['Description'].values:
            if label=='ProductSubcategory':
                submitdf = pd.DataFrame([{'Description': v, 'AlternativeDescription': '', 'ProductCategory': None, 
                                          'Active': True, 'OptimisticLockField': None}])
                # submitdf.to_sql(label, schema='%s.dbo' % DB_NAME, con=engine, if_exists='append', index=False)
                submitdf.to_sql(label, con=engine, if_exists='append', index=False)
            else:
                submitdf = pd.DataFrame([{'Description': v, 'AlternativeDescription': '', 'Active': True,
                                          'OptimisticLockField': None}])
                # submitdf.to_sql(label, schema='%s.dbo' % DB_NAME, con=engine, if_exists='append', index=False)
                submitdf.to_sql(label, con=engine, if_exists='append', index=False)
            

In [121]:
if v not in dfDict[label + '_DB']['Description'].values:
    print(label, v, label + '_DB')
    submitdf = pd.DataFrame([{'Description': v, 'AlternativeDescription': '', 'Active': True, 'OptimisticLockField': None}])
    submitdf.to_sql(label, con=engine, if_exists='append', index=False)
submitdf

Fit REGULAR FIT Fit_DB


,Description,AlternativeDescription,Active,OptimisticLockField
0,REGULAR FIT,,True,None


In [136]:
labelsUnique.keys()

dict_keys(['ProductCategory', 'ProductSubcategory', 'Length', 'Sleeve', 'CollarDesign', 'NeckDesign', 'Fit'])

In [124]:
labelsNRG

,ProductCategory,ProductSubcategory,Gender,LifeStage,TrendTheme,InspirationBackground,Length,Sleeve,CollarDesign,NeckDesign,Fit,SampleManufacturer,ProductionManufacturer,Colors
0,SET,SHORT SET,Man-Boy,Infant,BASIC LINE,CHILDRENSALON,SHORT,SHORT SLEEVE,POLO COLLAR,ROUND NECK,REGULAR FIT,APPLE APPARELSS,APPLE APPARELSS,"ΓΚΡΙ_175,175,175"
1,BERMUDAS-SHORTS,SHORTS,Woman-Girl,0 to 5,BEACH & SPORT,PINTEREST,LONG,LONG SLEEVE,SHIRT COLLAR,COLLAR,CARGO,NaN,NaN,ΕΜΠΡΙΜΕ_
2,BLOUSES,BLOUSE POLO SHORT SLEEVE,,1 to 5,BEACH AND SPORT,STYLE RIGHT,MEDIUM,TURN UP SLEEVE,FLAT KNITTED RIB,TURTLENECK,RELAXED FIT,JESSE GARMENTS LTD,JESSE GARMENTS LTD,"ΛΕΥΚΟ_255,255,255"
3,TROUSERS,JEANS,,1 to 16,COOL GUYS,SHUTTERSTOCK,KNEE,SLEEVELESS,MAO COLLAR,HOODED,SLIM FIT,BEBESAN TEKS.SAV.VE DIS TIC.LTD.STI.,BEBESAN TEKS.SAV.VE DIS TIC.LTD.STI.,"ΜΠΛΕ ΤΖΗΝ_6,61,121"
4,ROMPER,INFANT ROMPER,,6 to 16,FREE LIFE,PITTI IMAGINE,CAPRI,RAGLAN SLEEVE,STAND UP COLLAR,V NECK,CHINOS,XIAMEN MICROUNION IND.,MINIDUNYA TEKSTIL HAKNUR BEBE,"ΜΑΡΕΝ_021,021,053"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ΜΠΕΖ_189,174,136,ΣΚΟΥΡΟ ΜΠΛΕ_046,095,255"
331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ΚΟΚΚΙΝΟ_242,011,046,ΜΑΡΕΝ_021,021,053,ΡΟΖ_255,..."
332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ΡΟΖ_255,128,192"
333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ΜΠΛΕ_28,28,255,ΣΟΜΟΝ_255,166,166"


In [132]:
labelsDF

,Oid,ProductCategory,ProductSubcategory,Length,Sleeve,CollarDesign,NeckDesign,Fit
0,774,None,None,None,None,None,None,RELAXED FIT
1,775,None,None,None,None,None,None,RELAXED FIT
2,776,None,None,None,None,None,None,RELAXED FIT
3,777,None,None,None,None,None,None,REGULAR FIT
4,778,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
347,1121,None,None,None,None,None,None,None
348,1122,None,None,None,None,None,None,None
349,1123,None,None,None,None,None,None,None
350,1124,None,None,None,None,None,None,None


In [134]:
labelsDF.loc[labelsDF['ProductCategory'].notnull(), ['ProductCategory', 'ProductSubcategory']]

,ProductCategory,ProductSubcategory
20,SET,SET
21,SET,"SET,TOP"
23,SKIRT,SKIRT
24,SET,"SET,JOGGERS,TOP"
40,DRESS,DRESS
42,DRESS,DRESS
88,TROUSERS,TROUSERS
94,DRESS,DRESS
97,DRESS,DRESS
98,SET,"SET,T-SHIRT"


In [ ]:
submitdf = pd.DataFrame(
            [{'Product': prdno, 'ReferenceOrder': referenceOrder, 'TrendingOrder': trendOrder, 'Price': price, 'OptimisticLockField': None}])
        # submitdf.to_sql("ProductHistory", schema='%s.dbo' % DB_NAME, con=ENGINE, if_exists='append', index=False)
        submitdf.to_sql("ProductHistory", con=ENGINE, if_exists='append', index=False)

In [96]:
#Read from database the labels 
# for name in (constants.NRGATTRIBUTES + constants.DEEPFASHIONATTRIBUTES):
dfDict = {}
for name in (constants.NRGATTRIBUTES):
    dfDict[str(name)+'_DB'] = pd.read_sql_query(constants.SELECTQUERY + '\"%s\"' % str(name), engine)
dfDict

{'ProductCategory_DB': Empty DataFrame
 Columns: [Oid, CreatedBy, UpdatedBy, CreatedOn, UpdatedOn, Description, AlternativeDescription, Active, Ordering, OptimisticLockField]
 Index: [], 'ProductSubcategory_DB': Empty DataFrame
 Columns: [Oid, CreatedBy, UpdatedBy, CreatedOn, UpdatedOn, Description, AlternativeDescription, Active, Ordering, ProductCategory, optimisticlockfield]
 Index: [], 'Length_DB':    Oid CreatedBy UpdatedBy           CreatedOn           UpdatedOn  \
 0   -1                     2020-10-17 08:54:00 2020-10-17 08:54:00   
 1    1                     2020-10-17 08:54:00 2020-10-17 08:54:00   
 2    2                     2020-10-17 08:54:00 2020-10-17 08:54:00   
 3    3                     2020-10-17 08:54:00 2020-10-17 08:54:00   
 4    4                     2020-10-17 08:54:00 2020-10-17 08:54:00   
 5    5                     2020-10-17 08:54:00 2020-10-17 08:54:00   
 6    6                     2020-10-17 08:54:00 2020-10-17 08:54:00   
 
      Description Alterna

In [101]:
assosDF.loc[assosDF['CollarDesign'].notnull(), 'Fit']

Series([], Name: Fit, dtype: object)